# YOLOv3

Source: https://machinelearningspace.com/yolov3-tensorflow-2-part-1/

## Arhitektura

YOLOv3 je sacinjen od 53 sloja nazvanim Darknet-53.

<figure style="display: inline-block">
  <img src="img/yolo_structure.png" width="400" height="500">
  <figcaption style="text-align: center"></figcaption>
</figure>

## Princip rada

YOLOv3 secka sliku na SxS resetku i predvidja granicne okvire i verovatnocu klase za svaku resetku. Za svaku resetku se predvidja B granicnih okvira i C verovatnoca klase za objekte <b>ciji se centar nalazi u resetci</b>. Svaka resetka ima (5 + C) atributa. Broj 5 predstavlja atribute granicnog okvira a to su: koordinate centra $(b_x,b_y)$ i oblik $(b_h, b_w)$ granicnog okvira i objektnost. Vrednost C je broj klasa. Objektnost predstavlja koliko je model siguran da okvir sadrzi objekat.<br>
YOLOv3 kreira 3D tensor oblika [S, S, B * (5 + C)].

<figure style="display: inline-block">
  <img src="img/yolo_work.png" width="800" height="700">
  <figcaption style="text-align: center"></figcaption>
</figure>

## Anchor Box algoritam

Problem koji zelimo da resimo je kada se u resetci nalazi vise od jednog centra objekta. To znaci da imamo vise objekta koji se preklapaju. Da bi prevazisao ovaj problem YOLOv3 koristi 3 drugacija anchor boxa za svaku skalu detekcije.

Anchor boxovi su set predefinisanih granicnih okvira odredjene visine i sirine koje koristimo da modelujemo drugaciju skalu i ascept ratio objekata koje zelimo da detektujemo.

## Predvidjanje kroz druge skale

YOLOv3 pravi detekcije u 3 drugacije skale da bi se prilagodio drugacijim velicima objekta tako sto koristi korake od 32, 16 i 8. To znaci da kada bi uneli sliku rezolucije 416x416, YOLOv3 bi pravio detekcija na velicinama 13x13, 26x26 i 52x52. 

<figure style="display: inline-block">
  <img src="img/yolo_scaling.png" width="800" height="600">
  <figcaption style="text-align: center"></figcaption>
</figure>

## Predvidjanje granicnih okvira

Za svaki granicni okvir, YOLO predvidja 3 koordinate $t_x, t_y, t_w, t_h$.Vrednosti| $t_x, t_y$ su koordinate centra granicnog okvira relativno na resetku gde se centar nalazi, $t_w, t_h$ su sirina i visina okvira.

Konacno predvidjanje okvira se dobija preko formule:

$b_x = \sigma(t_x) + c_x$ <br>
$b_y = \sigma(t_y) + c_y$ <br>
$b_w = p_w*e^{t_w}$ <br>
$b_h = p_h*e^{t_h}$

Vrednosti $p_w, p_h$ su sirina i visina anchor boxa, a $c_x, c_y$ koordinate resetke.

## Odbacivanje ne maksimalnih

1. Na izlaz CNN se dodaje dopunska objektnost koja mora koristiti sigmoidnu funkciju i gubitak binarna unakrsna entropija. Zatim se odbacuju svi granicni okviri cija vrednost objektnost je manja od odredjenog praga: tako ce nestati svi granicni okviri koji ne sadrze cvetove.
2. Nadje se granicni okvir sa najvisom vrednoscui ovjektnosti i odbace se svi drugi granicni okviri koji se znacajno prekrivaju sa njim (IoU > 60%).
3. Korak 2 se ponavlja sve dok vise ne bude granicnih okvira za odbacivanje

## Implementacija

Implementiranje Darkneta i dodavanje YOLO dela mreze je jednostavno do NMS i bounding box dela. Arhitektura cele mreze je sumirana u sledece dve slike, a NMS i bounding box cu pokusati da objasnim.

<figure style="display: inline-block">
  <img src="img/darknet_structure.png" width="500" height="350">
  <figcaption style="text-align: center"></figcaption>
</figure>
<figure style="display: inline-block">
  <img src="img/yolo_architecture.png" width="800" height="350">
  <figcaption style="text-align: center"></figcaption>
</figure>

Prva funkcija odredjuje bounding boxove za predikcije mreze, to jest dekodira predikcije mreze.

In [ ]:
import tensorflow as tf

def yolo_boxes(pred, anchors, classes):
  # uzimamo vrednosti iz predikcije koja je oblika
  # [[box_x, box_y, box_w, box_h, objectness, class_probs...],
  # [box_x, box_y, box_w, box_h, objectness, class_probs...]...]
  # axis=-1 znaci da zelimo poslednju dimenziju u ovom slucaju po redovima
  grid_size = tf.shape(pred)[1:3]
  box_xy, box_wh, objectness, class_probs = tf.split(
    pred, (2, 2, 1, classes), axis=-1
  )
  
  # koristimo sigmoidnu funkciju na svakoj vrednosti da bi je skalirali
  # u opsed od 0 do 1
  # posle ovog koraka imamo koordinate centra relativno na resetku
  box_xy = tf.sigmoid(box_xy)
  objectness = tf.sigmoid(objectness)
  class_probs = tf.sigmoid(class_probs)
  pred_box = tf.concat((box_xy, box_wh), axis=-1)
  
  # funkcija meshgrid vraca tensor kooridnata resetke gde mozemo videti
  # koordinate resetke (i,j) indeksiranjem
  grid = tf.meshgrid(tf.range(grid_size[1]), tf.range(grid_size[0]))
  grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2) # [gx, gy, 1, 2]
  
  # svim koordinatama dodajemo koordinate resetke gde se nalaze
  # i tako dobijamo koordinate kutije relativno na gornji levi ugao, tj. pocetak
  box_xy = (box_xy + tf.cast(grid, tf.float32)) / \
    tf.cast(grid_size, tf.float32)
    
  # kalkulisemo sirine i visine svih granicnih kutija
  box_wh = tf.exp(box_wh) * anchors
    
  # da bismo dobili koordinate levog gornjeg i donjeg desnog ugla
  # radimo sledece kalkulacije
  box_x1y1 = box_xy - box_wh / 2
  box_x2y2 = box_xy + box_wh / 2
  bbox = tf.concat([box_x1y1, box_x2y2], axis=-1)
  
  return bbox, objectness, class_probs, pred_box

In [ ]:
def yolo_nms(outputs, anchors, masks, classes):
  # prvi korak je da spojimo sve vrednosti koje se nalaze na izlazu mreze
  b, c, t = [], [], []
  
  for o in outputs:
    b.append(tf.reshape(o[0], (tf.shape(o[0])[0], -1, tf.shape(o[0])[-1])))
    c.append(tf.reshape(o[1], (tf.shape(o[1])[0], -1, tf.shape(o[1])[-1])))
    t.append(tf.reshape(o[2], (tf.shape(o[2])[0], -1, tf.shape(o[2])[-1])))
    
  bbox = tf.concat(b, axis=1)
  confidence = tf.concat(c, axis=1)
  class_probs = tf.concat(t, axis=1) 
    
  if classes == 1:
    scores = confidence
  else:
    scores = confidence * class_probs
    
  dscores = tf.squeeze(scores, axis=0)
  scores = tf.reduce_max(dscores, [1])
  bbox = tf.reshape(bbox, (-1, 4))
  classes = tf.argmax(dscores, 1)
  selected_indicies, selected_scores = tf.image.non_max_suppression_with_scores(
    boxes=bbox,
    scores=scores,
    max_output_size=max_boxes,
    iou_threshold=iou_threshold,
    score_threshold=score_threshold,
    soft_nms_sigma=0.5
  ) 
  
  num_valid_nms_boxes = tf.shape(selected_indicies)[0]
  
  selected_indicies = tf.concat([selected_indicies, tf.zeros(max_boxes - num_valid_nms_boxes, tf.int32)], 0)
  selected_scores = tf.concat([selected_scores, tf.zeros(max_boxes - num_valid_nms_boxes, tf.float32)], -1)

  boxes = tf.gather(bbox, selected_indicies)
  boxes = tf.expand_dims(boxes, axis=0)
  scores = selected_scores
  scores = tf.expand_dims(scores, axis=0)
  classes = tf.gather(classes, selected_indicies)
  classes = tf.expand_dims(classes, axis=0)
  valid_detections = num_valid_nms_boxes 
  valid_detections = tf.expand_dims(valid_detections, axis=0)
  
  return boxes, scores, classes, valid_detections   